## Проект по Автобрее 3 курс 1 модуль

In [1]:
#импортируем нужные библиотеки
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import nltk
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
m = Mystem()
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to C:\Users\Abina
[nltk_data]     Kukanova\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


В файле anecdotes.txt лежат тексты анекдотов и ссылки на источники. Разделим тексты на предложения, и каждый пример и ссылку на источник добавим в словарь. После этого создадим список, куда добавим лемматизированные предложения из данного словаря. После создадим кортеж из ключей, значений словаря и элементов списка. Запишем элементы кортежа в базу данных: первый столбец - предложения, второй столбец - лематизированные предложения, третий столбец - ссылки на источник. 

In [4]:
sent_source = {}

with open("anecdotes.txt", "r", encoding="utf-8") as file:
    for source in file.read().split("\n\n\n")[1:]:
        for part in source.split("\n\n")[1:]:
            link = source.split("\n\n")[0]
            for sent in sent_tokenize(part):
                sent_source[sent] = link

In [5]:
# лемматизируем текст
def lemmatize(text):
  lemmas = m.lemmatize(text)
  lemm_sample = ''.join(lemmas)
  return lemm_sample

In [ ]:
spisok = []
for sent in sent_source:
    sent = lemmatize(sent)
    sent = sent.rstrip()
    spisok.append(lemmatize(sent))

In [36]:
keys = list(sent_source.keys())
values = list(sent_source.values())
myList = list(zip(keys, values))

lemm_tuple = tuple(spisok)

In [39]:
result = []
for item, key in zip(myList, lemm_tuple):
    new_item = item + (key,)
    result.append(new_item)

In [2]:
import sqlite3

conn = sqlite3.connect("db_anectodes.db")
cur = conn.cursor()

In [43]:
cur.execute(
    """
CREATE TABLE IF NOT EXISTS texts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    text TEXT,
    lemm_text TEXT,
    source TEXT

)
"""
)

conn.commit()

In [44]:
def exception(err_sent, error):
    with open("error.txt", "a+") as file:
        data = f"{error}: {err_sent}"
        file.write(data)
        file.write("\n")

In [45]:
def write_to_db(sentence, lemm_sentence, url):
    cur.execute(
        """
        INSERT INTO texts
            (text, lemm_text, source) VALUES (?, ?, ?)
        """,
        (
            sentence,
            lemm_sentence,
            url
        ),
    )
    conn.commit()

In [46]:
for res in result:
    try:
        write_to_db(res[0], res[2], res[1])
    except Exception as e:
        exception(sent, e)

Используем граммемы морфологического анализатора pymorphy и создадим кортеж со списком граммем, которые используются в данном модуле. Список граммем находится здесь - https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html

In [3]:
pos_taggers = ('NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF',
               'PRTS', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP',
               'CONJ', 'PRCL', 'INTJ')

Создадим функцию search, которая на вход принимает наши запросы, превращает их в словарь и проверяет на количество элементов. Если в запросе один элемент и нет плюса, то вызываем функцию first_function, если есть один элемент и есть плюс, то вызываем функцию second question. Если в запросе два элемента, то вызываем функцию third_function, если в запросе три элемента, то вызываем функцию forth_function. 

Функция first_function проверяет, если элемент с двойными кавычки или без, или это часть речи. Если это элемент с двойнымм кавычками, то убираем кавычки и ищем данное слово в тексте, записываем предложения и источник в словарь. Если это часть речи, то ищем слова с данной частью речи. Если это слово без кавычек, то находим лемму данного слова в столбце с лемматизированными предложениями и записываем предложения из первого столбца в словарь. Если это слово+тег, то ищем в тексте слово в данной словоформе, имеющей данный тег. Функция second_function разделяет запрос через плюс, записывает в список, потом ищет данное слово (это будет конкретная словоформа) в предложении и проверяет часть речи, потом записываем в словарь. Функция third_function проверяет первое слово, если оно в двойных кавычках, если это часть речи, с плюсом, или это слово без кавычек. Проверив первое слово, дальше также проверяет второе слово, ищет два этих слова в предложениях и записывает в словарь. Функция forth_function работает также как и третья функция, но после второго слова еще проверяет третье, ищет все три слова в предложениях и записываем в словарь. 

In [4]:
def first_function(word):
    res_sentences = {}
    cur.execute("SELECT * FROM texts")
    rows = cur.fetchall()
    if re.findall(r'"[^\s]+"', word):
        word = re.sub(r'[^А-яЁё]+', ' ', word)
        for row in rows:
            if word in row[1].lower():
                res_sentences[row[1]] = row[3]

    else:
        if word in pos_taggers:
            for row in rows:
                tokens = word_tokenize(row[1].lower())
                for token in tokens:
                    if word in morph.parse(token)[0].tag:
                        if row[1] not in res_sentences:
                            res_sentences[row[1]] = row[3]
        else:
            lemm_word = morph.parse(word)[0].normal_form
            for row in rows:
                if lemm_word in row[2]:
                    res_sentences[row[1]] = row[3]
    return res_sentences

In [5]:
def second_function(word):
    word_and_pos = word.split('+')
    cur.execute("SELECT * FROM texts")
    rows = cur.fetchall()
    res_sentences = {}
    for row in rows:
        tokens = word_tokenize(row[1].lower())
        for token in tokens:
            if token == word_and_pos[0] and word_and_pos[1] in morph.parse(token)[0].tag:
                res_sentences[row[1]] = row[3]
    return res_sentences

In [6]:
def third_function(word_list):
    cur.execute("SELECT * FROM texts")
    rows = cur.fetchall()
    res_sentences = {}
    if re.findall(r'"[^\s]+"', word_list[0]):
        word_list[0] = re.sub(r'[^А-яЁё]+', '', word_list[0])
        for row in rows:
            tokens = word_tokenize(row[1].lower())
            for index, token in enumerate(tokens):
                if index < (len(tokens) - 1):
                    if tokens[index] == word_list[0]:
                        if re.findall(r'"[^\s]+"', word_list[1]):
                            word_list[1] = re.sub(r'[^А-яЁё]+', '', word_list[1])
                            if tokens[index+1] == word_list[1]:
                                res_sentences[row[1]] = row[3]
                        else:
                            if word_list[1] in pos_taggers:
                                if word_list[1] in morph.parse(tokens[index+1])[0].tag:
                                    if row[1] not in res_sentences:
                                        res_sentences[row[1]] = row[3]
                            elif '+' in word_list[1]:
                                if tokens[index+1] == word_list[1].split('+')[0] and word_list[1].split('+')[1] in morph.parse(tokens[index+1])[0].tag:
                                    res_sentences[row[1]] = row[3]
                            else:
                                lemm_word = morph.parse(word_list[1])[0].normal_form
                                if morph.parse(tokens[index+1])[0].normal_form == lemm_word:
                                    res_sentences[row[1]] = row[3]
    else:
        if word_list[0] in pos_taggers:
            for row in rows:
                tokens = word_tokenize(row[1].lower())
                for index, token in enumerate(tokens):
                    if index < (len(tokens) - 1):
                        if word_list[0] in morph.parse(tokens[index])[0].tag:
                            if re.findall(r'"[^\s]+"', word_list[1]):
                                word_list[1] = re.sub(r'[^А-яЁё]+', '', word_list[1])
                                if tokens[index+1] == word_list[1]:
                                    res_sentences[row[1]] = row[3]
                            else:
                                if word_list[1] in pos_taggers:
                                    if word_list[1] in morph.parse(tokens[index+1])[0].tag:
                                        if row[1] not in res_sentences:
                                            res_sentences[row[1]] = row[3]
                                if '+' in word_list[1]:
                                    if tokens[index+1] == word_list[1].split('+')[0] and word_list[1].split('+')[1] in morph.parse(tokens[index+1])[0].tag:
                                        res_sentences[row[1]] = row[3]
                                else:
                                    lemm_word = morph.parse(word_list[1])[0].normal_form
                                    if morph.parse(tokens[index+1])[0].normal_form == lemm_word:
                                        res_sentences[row[1]] = row[3]
        if '+' in word_list[0]:
            for row in rows:
                tokens = word_tokenize(row[1].lower())
                for index, token in enumerate(tokens):
                    if index < (len(tokens) - 1):
                        if tokens[index] == word_list[0].split('+')[0] and word_list[0].split('+')[1] in morph.parse(tokens[index])[0].tag:
                            if re.findall(r'"[^\s]+"', word_list[1]):
                                word_list[1] = re.sub(r'[^А-яЁё]+', '', word_list[1])
                                if tokens[index+1] == word_list[1]:
                                    res_sentences[row[1]] = row[3]
                            else:
                                if word_list[1] in pos_taggers:
                                    if word_list[1] in morph.parse(tokens[index+1])[0].tag:
                                        if row[1] not in res_sentences:
                                            res_sentences[row[1]] = row[3]
                                if '+' in word_list[1]:
                                    if tokens[index+1] == word_list[1].split('+')[0] and word_list[1].split('+')[1] in morph.parse(tokens[index+1])[0].tag:
                                        res_sentences[row[1]] = row[3]
                                else:
                                    lemm_word = morph.parse(word_list[1])[0].normal_form
                                    if morph.parse(tokens[index+1])[0].normal_form == lemm_word:
                                        res_sentences[row[1]] = row[3]
        else:
            lemm_word = morph.parse(word_list[0])[0].normal_form
            for row in rows:
                tokens = word_tokenize(row[1].lower())
                for index, token in enumerate(tokens):
                    if index < (len(tokens) - 1):
                        if morph.parse(tokens[index])[0].normal_form == lemm_word:
                            if re.findall(r'"[^\s]+"', word_list[1]):
                                word_list[1] = re.sub(r'[^А-яЁё]+', '', word_list[1])
                                if tokens[index+1] == word_list[1]:
                                    res_sentences[row[1]] = row[3]
                            else:
                                if word_list[1] in pos_taggers:
                                    if word_list[1] in morph.parse(tokens[index+1])[0].tag:
                                        if row[1] not in res_sentences:
                                            res_sentences[row[1]] = row[3]
                                if '+' in word_list[1]:
                                    if tokens[index+1] == word_list[1].split('+')[0] and word_list[1].split('+')[1] in morph.parse(tokens[index+1])[0].tag:
                                        res_sentences[row[1]] = row[3]
                                else:
                                    lemm_word = morph.parse(word_list[1])[0].normal_form
                                    if morph.parse(tokens[index+1])[0].normal_form == lemm_word:
                                        res_sentences[row[1]] = row[3]
    return res_sentences

In [7]:
def forth_function(word_list):
    cur.execute("SELECT * FROM texts")
    rows = cur.fetchall()
    res_sentences = {}
    if re.findall(r'"[^\s]+"', word_list[0]):
        word_list[0] = re.sub(r'[^А-яЁё]+', '', word_list[0])
        for row in rows:
            tokens = word_tokenize(row[1].lower())
            for index, token in enumerate(tokens):
                if index < (len(tokens) - 1):
                    if tokens[index] == word_list[0]:
                        if re.findall(r'"[^\s]+"', word_list[1]):
                            word_list[1] = re.sub(r'[^А-яЁё]+', '', word_list[1])
                            if tokens[index+1] == word_list[1]:
                                if re.findall(r'"[^\s]+"', word_list[2]):
                                    word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                    if tokens[index+2] == word_list[2]:
                                        res_sentences[row[1]] = row[3]
                                else:
                                    if word_list[2] in pos_taggers:
                                        if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                            res_sentences[row[1]] = row[3]
                                    if "+" in word_list[2]:
                                        if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                            res_sentences[row[1]] = row[3]
                                    else:
                                        lemm_word = morph.parse(word_list[2])[0].normal_form
                                        if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                            res_sentences[row[1]] = row[3]
                        else:
                            if word_list[1] in pos_taggers:
                                if word_list[1] in morph.parse(tokens[index+1])[0].tag:
                                    if re.findall(r'"[^\s]+"', word_list[2]):
                                        word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                        if tokens[index+2] == word_list[2]:
                                            res_sentences[row[1]] = row[3]
                                    else:
                                        if word_list[2] in pos_taggers:
                                            if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                res_sentences[row[1]] = row[3]
                                        if '+' in word_list[2]:
                                            if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            lemm_word = morph.parse(word_list[2])[0].normal_form
                                            if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                res_sentences[row[1]] = row[3]
                            if '+' in word_list[1]:
                                if tokens[index+1] == word_list[1].split('+')[0] and word_list[1].split('+')[1] in morph.parse(tokens[index+1])[0].tag:
                                    if re.findall(r'"[^\s]+"', word_list[2]):
                                        word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                        if tokens[index+2] == word_list[2]:
                                            res_sentences[row[1]] = row[3]
                                    else:
                                        if word_list[2] in pos_taggers:
                                            if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                res_sentences[row[1]] = row[3]
                                        if '+' in word_list[2]:
                                            if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            lemm_word = morph.parse(word_list[2])[0].normal_form
                                            if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                res_sentences[row[1]] = row[3]
                            else:
                                lemm_word = morph.parse(word_list[1])[0].normal_form
                                if morph.parse(tokens[index+1])[0].normal_form == lemm_word:
                                    if re.findall(r'"[^\s]+"', word_list[2]):
                                        word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                        if tokens[index+2] == word_list[2]:
                                            res_sentences[row[1]] = row[3]
                                    else:
                                        if word_list[2] in pos_taggers:
                                            if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                res_sentences[row[1]] = row[3]
                                        if '+' in word_list[2]:
                                            if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            lemm_word = morph.parse(word_list[2])[0].normal_form
                                            if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                res_sentences[row[1]] = row[3]
    else:
        if word_list[0] in pos_taggers:
            for row in rows:
                tokens = word_tokenize(row[1].lower())
                for index, token in enumerate(tokens):
                    if index < (len(tokens) - 1):
                        if word_list[0] in morph.parse(tokens[index])[0].tag:
                            if re.findall(r'"[^\s]+"', word_list[1]):
                                word_list[1] = re.sub(r'[^А-яЁё]+', '', word_list[1])
                                if tokens[index+1] == word_list[1]:
                                    if re.findall(r'"[^\s]+"', word_list[2]):
                                        word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                        if tokens[index+2] == word_list[2]:
                                            res_sentences[row[1]] = row[3]
                                    else:
                                        if word_list[2] in pos_taggers:
                                            if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                res_sentences[row[1]] = row[3]
                                        if '+' in word_list[2]:
                                            if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            lemm_word = morph.parse(word_list[2])[0].normal_form
                                            if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                res_sentences[row[1]] = row[3]
                            else:
                                if word_list[1] in pos_taggers:
                                    if word_list[1] in morph.parse(tokens[index+1])[0].tag:
                                        if re.findall(r'"[^\s]+"', word_list[2]):
                                            word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                            if tokens[index+2] == word_list[2]:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            if word_list[2] in pos_taggers:
                                                if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            if '+' in word_list[2]:
                                                if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            else:
                                                lemm_word = morph.parse(word_list[2])[0].normal_form
                                                if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                    res_sentences[row[1]] = row[3]
                                if '+' in word_list[1]:
                                    if tokens[index+1] == word_list[1].split('+')[0] and word_list[1].split('+')[1] in morph.parse(tokens[index+1])[0].tag:
                                        if re.findall(r'"[^\s]+"', word_list[2]):
                                            word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                            if tokens[index+2] == word_list[2]:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            if word_list[2] in pos_taggers:
                                                if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            if '+' in word_list[2]:
                                                if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            else:
                                                lemm_word = morph.parse(word_list[2])[0].normal_form
                                                if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                    res_sentences[row[1]] = row[3]
                                else:
                                    lemm_word = morph.parse(word_list[1])[0].normal_form
                                    if morph.parse(tokens[index+1])[0].normal_form == lemm_word:
                                        if re.findall(r'"[^\s]+"', word_list[2]):
                                            word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                            if tokens[index+2] == word_list[2]:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            if word_list[2] in pos_taggers:
                                                if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            if '+' in word_list[2]:
                                                if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            else:
                                                lemm_word = morph.parse(word_list[2])[0].normal_form
                                                if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                    res_sentences[row[1]] = row[3]
        if '+' in word_list[0]:
            for row in rows:
                tokens = word_tokenize(row[1].lower())
                for index, token in enumerate(tokens):
                    if index < (len(tokens) - 1):
                        if tokens[index] == word_list[0].split('+')[0] and word_list[0].split('+')[1] in morph.parse(tokens[index])[0].tag:
                            if re.findall(r'"[^\s]+"', word_list[1]):
                                word_list[1] = re.sub(r'[^А-яЁё]+', '', word_list[1])
                                if tokens[index+1] == word_list[1]:
                                    if re.findall(r'"[^\s]+"', word_list[2]):
                                        word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                        if tokens[index+2] == word_list[2]:
                                            res_sentences[row[1]] = row[3]
                                    else:
                                        if word_list[2] in pos_taggers:
                                            if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                res_sentences[row[1]] = row[3]
                                        if '+' in word_list[2]:
                                            if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            lemm_word = morph.parse(word_list[2])[0].normal_form
                                            if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                res_sentences[row[1]] = row[3]
                            else:
                                if word_list[1] in pos_taggers:
                                    if word_list[1] in morph.parse(tokens[index+1])[0].tag:
                                        if re.findall(r'"[^\s]+"', word_list[2]):
                                            word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                            if tokens[index+2] == word_list[2]:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            if word_list[2] in pos_taggers:
                                                if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            if '+' in word_list[2]:
                                                if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            else:
                                                lemm_word = morph.parse(word_list[2])[0].normal_form
                                                if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                    res_sentences[row[1]] = row[3]
                                if '+' in word_list[1]:
                                    if tokens[index+1] == word_list[1].split('+')[0] and word_list[1].split('+')[1] in morph.parse(tokens[index+1])[0].tag:
                                        if re.findall(r'"[^\s]+"', word_list[2]):
                                            word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                            if tokens[index+2] == word_list[2]:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            if word_list[2] in pos_taggers:
                                                if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            if '+' in word_list[2]:
                                                if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            else:
                                                lemm_word = morph.parse(word_list[2])[0].normal_form
                                                if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                    res_sentences[row[1]] = row[3]
                                else:
                                    lemm_word = morph.parse(word_list[1])[0].normal_form
                                    if morph.parse(tokens[index+1])[0].normal_form == lemm_word:
                                        if re.findall(r'"[^\s]+"', word_list[2]):
                                            word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                            if tokens[index+2] == word_list[2]:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            if word_list[2] in pos_taggers:
                                                if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            if '+' in word_list[2]:
                                                if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            else:
                                                lemm_word = morph.parse(word_list[2])[0].normal_form
                                                if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                    res_sentences[row[1]] = row[3]
        else:
            lemm_word = morph.parse(word_list[0])[0].normal_form
            for row in rows:
                tokens = word_tokenize(row[1].lower())
                for index, token in enumerate(tokens):
                    if index < (len(tokens) - 1):
                        if morph.parse(tokens[index])[0].normal_form == lemm_word:
                            if re.findall(r'"[^\s]+"', word_list[1]):
                                word_list[1] = re.sub(r'[^А-яЁё]+', '', word_list[1])
                                if tokens[index+1] == word_list[1]:
                                    if re.findall(r'"[^\s]+"', word_list[2]):
                                        word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                        if tokens[index+2] == word_list[2]:
                                            res_sentences[row[1]] = row[3]
                                    else:
                                        if word_list[2] in pos_taggers:
                                            if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                res_sentences[row[1]] = row[3]
                                        if '+' in word_list[2]:
                                            if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            lemm_word = morph.parse(word_list[2])[0].normal_form
                                            if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                res_sentences[row[1]] = row[3]
                            else:
                                if word_list[1] in pos_taggers:
                                    if word_list[1] in morph.parse(tokens[index+1])[0].tag:
                                        if re.findall(r'"[^\s]+"', word_list[2]):
                                            word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                            if tokens[index+2] == word_list[2]:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            if word_list[2] in pos_taggers:
                                                if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            if '+' in word_list[2]:
                                                if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            else:
                                                lemm_word = morph.parse(word_list[2])[0].normal_form
                                                if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                    res_sentences[row[1]] = row[3]
                                if '+' in word_list[1]:
                                    if tokens[index+1] == word_list[1].split('+')[0] and word_list[1].split('+')[1] in morph.parse(tokens[index+1])[0].tag:
                                        if re.findall(r'"[^\s]+"', word_list[2]):
                                            word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                            if tokens[index+2] == word_list[2]:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            if word_list[2] in pos_taggers:
                                                if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            if '+' in word_list[2]:
                                                if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            else:
                                                lemm_word = morph.parse(word_list[2])[0].normal_form
                                                if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                    res_sentences[row[1]] = row[3]
                                else:
                                    lemm_word = morph.parse(word_list[1])[0].normal_form
                                    if morph.parse(tokens[index+1])[0].normal_form == lemm_word:
                                        if re.findall(r'"[^\s]+"', word_list[2]):
                                            word_list[2] = re.sub(r'[^А-яЁё]+', '', word_list[2])
                                            if tokens[index+2] == word_list[2]:
                                                res_sentences[row[1]] = row[3]
                                        else:
                                            if word_list[2] in pos_taggers:
                                                if word_list[2] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            if '+' in word_list[2]:
                                                if tokens[index+2] == word_list[2].split('+')[0] and word_list[2].split('+')[1] in morph.parse(tokens[index+2])[0].tag:
                                                    res_sentences[row[1]] = row[3]
                                            else:
                                                lemm_word = morph.parse(word_list[2])[0].normal_form
                                                if morph.parse(tokens[index+2])[0].normal_form == lemm_word:
                                                    res_sentences[row[1]] = row[3]
    return res_sentences

In [8]:
def search(input_request):
    input_list = input_request.split()
    if len(input_list) == 1:
        for input_word in input_list:
            if '+' in input_word:
                return(second_function(input_word))
            else:
                return(first_function(input_word))
    elif len(input_list) == 2:
        return third_function(input_list)
    elif len(input_list) == 3:
        return forth_function(input_list)

Чтобы ввести запросы, нужно вызвать функцию search и ввести на вход наш запрос. Приведем примеры какие запросы наша функция может обрабатывать. Чтобы проверить первую функцию, то возьмем, например, такие запросы "идет", мужик и ADVB. Чтобы проверить вторую функцию, то введем такой запрос спрашивает+VERB. Чтобы проверить третью функцию, то введем такие запросы. Вы можете взять любые другие вопросы для третьей функции, мы взяли эти просто для примера:
- "прекрасный" день
- великолепная NOUN
- "баре" "хогсмида"
- больнице "звонит"
- ADJF NOUN
- VERB мужик
- преподаватель+NOUN VERB
- ADJF "время"
- мужик+NOUN убрал+VERB

Для четвертой функции, например, возьмем такие запросы:
- расскажите+VERB о "героине"
- всего "два" посетителя+NOUN
- баре хогсмида всего
- "бар" заходит+VERB лошадь
- NOUN VERB NOUN
- врач+NOUN "опять" раскрывает+VERB

Дальше будет проверка запросов и много инпутов

In [54]:
search('"идет"')

{'Он грязный, поэтому идет мыться.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Последним идет и этот.': 'https://www.anecdote.tv/no-category/6880-dlinnye-smeshnye-anekdoty-122-shtuki.html?ysclid=lnkqe5fh33111440614',
 'Оставшись одна ночевать в темном и мрачном здании и услышав подозрительный звук, героиня идет узнавать в чем дело, надев самое роскошное и соблазнительное белье.': 'https://shytok.net/anekdots/anekdots-pro-dlinnyeanekdoty.html',
 'Сын идет домой и говорит матери:\n- Мам, батя сказал, что б ты сварила пельменей и купила водки!': 'https://shytok.net/anekdots/anekdots-pro-dlinnyeanekdoty.html',
 'Сын ставит косу на место и идет обратно в поле и говорит отцу:\n- Бать, мама там пельменей сварила и водки купила, говорит что б мы домой шли!': 'https://shytok.net/anekdots/anekdots-pro-dlinnyeanekdoty.html',
 'Сын идет за водкой и по дороге заходит к соседу.': 'https://shytok.net/anekdots/anekdots-pro-dlinnyeanekdoty.html',
 '- 

In [55]:
search('мужик')

{'— Ну, как тебе объяснить… Видишь — вон два мужика идут, один грязный, другой чистый.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 '— Ну… Видишь, два мужика идут.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 '— Видишь… Два мужика идут, грязный и чистый.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Пошёл мужик к хирургу, там ему левое ухо отрезали.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Пошёл мужик к хирургу, там ему и правое ухо отрезали.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Тут же раздался гром небесный, спустился апостол, и привёл с собой страшного, просто жуткого мужика.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Пристегнув его к ней наручниками, апостол сказал:\n— Ты

In [56]:
search('ADVB')

{'В баре Хогсмида всего два посетителя, один спьяну спотыкается о ногу другого.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Первый говорит:\n— А ты, друг, откуда будешь?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 '—Ну, ничего себе!': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Чего у вас тут новенького?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Бармен:\n— Да ничего, вот близнецы Уизли опять нажрались…': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Через минуту опять подходит Чебурашка и спрашивает:\n— Гена, а у тебя апельсины есть?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Чебурашка если ты ещё раз спросишь про апельсины, я возьму молоток, гвозди и прибью тебя за уши к стене.': 'h

In [57]:
search('спрашивает+VERB')

{'Подходит Чебурашка и спрашивает:\n— Гена, а у тебя апельсины есть?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Через минуту опять подходит Чебурашка и спрашивает:\n— Гена, а у тебя апельсины есть?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Через минуту опять подходит Чебурашка и спрашивает:\n— Гена, а у тебя есть молоток?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Один другого спрашивает:\n— Слушай, а что такое логика?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Спрашивает:\n— Рит, ты чего?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Она рот открыла от удивления и спрашивает:\n— Боже, за что мне такое счастье?!': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Старший его спрашивае

In [58]:
search('"прекрасный" день')

{'— Потом ты будешь ловить ещё больше рыбы и сможешь купить себе несколько лодок, даже кораблей, и в один прекрасный день у тебя будет целая флотилия.': 'https://megapanoptikum.info/tags/длинные%20анекдоты/'}

In [59]:
search('великолепная NOUN')

{'У средневековых и даже первобытных жителей всегда блестящие, ухоженнные роскошные волосы и великолепные зубы.': 'https://shytok.net/anekdots/anekdots-pro-dlinnyeanekdoty.html'}

In [63]:
search('баре хогсмида всего')

{'В баре Хогсмида всего два посетителя, один спьяну спотыкается о ногу другого.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566'}

In [9]:
search('"баре" "хогсмида"')

{'В баре Хогсмида всего два посетителя, один спьяну спотыкается о ногу другого.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566'}

In [13]:
search('преподаватель+NOUN VERB')

{'Ничего не сказав, преподаватель садится на свое место и с ухмылкой ждет звонка.': 'https://www.anecdote.tv/no-category/6880-dlinnye-smeshnye-anekdoty-122-shtuki.html?ysclid=lnkqe5fh33111440614',
 'Преподаватель говорит:\n- Тема нашей сегодняшней лекции...\nТут стук в дверь и вваливается студент.': 'https://baneks.site',
 'Преподаватель устал слушать.': 'https://baneks.site'}

In [10]:
search('больнице "звонит"')

{'В больнице звонит телефон...\n- Доброе утро, я хотел бы справиться о здоровье больного Сидорова из\nшестой палаты.': 'https://vipanekdot.ru/anekdoty/dlinnye/page/2/'}

In [14]:
search('ADJF "время"')

{'Через некоторое время в этот же коридор вышла и Наташа, она встала возле соседнего окна и принялась всячески привлекать внимание поручика.': 'https://megapanoptikum.info/tags/длинные%20анекдоты/',
 'Через некоторое время подходит майор к полковнику и говорит:\n- Товарищ полковник, ходите, я вам классную загадку загадаю?': 'https://vseanekdoti.ucoz.ru/index/bolshie_i_dlinnye_anekdoty_sbornik_1/0-246?ysclid=lnmzcvstfy854784716'}

In [11]:
search('ADJF NOUN')

{'— А ты, в какую школу ходил?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'В это время в бар заходит третий клиент, говорит бармену:\n— Налей-ка мне кружечку.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Один другого спрашивает:\n— Слушай, а что такое логика?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 '— А что такое диалектика?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Осмотрел доктор мужа, отпустил, а жене его говорит:\n— Как мне ни больно об этом говорить, но ваш муж страдает смертельной болезнью.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Для того чтобы он жил вам нужно потакать ему во всех желаниях, привлекательно выглядеть, смотреть с ним его телепередачи, не забирать зарплату, разрешать пить пиво с друзьями и только в этом

In [12]:
search('VERB мужик')

{'Пошёл мужик к хирургу, там ему левое ухо отрезали.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Пошёл мужик к хирургу, там ему и правое ухо отрезали.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Тут с 10 го этажа недостроенного дома выпадает мужик и разбивается насмерть.': 'https://www.anecdote.tv/no-category/6880-dlinnye-smeshnye-anekdoty-122-shtuki.html?ysclid=lnkqe5fh33111440614',
 'Не успел мужик опомниться, как тигр сбивает его с ног и мчится дальше.': 'https://baneks.ru/?ysclid=lnkods4e8x989429626',
 'Сидит мужик в баре, пьет.': 'https://baneks.site',
 '– спрашивает мужик.': 'https://baneks.site',
 '– Я попрошу, чтобы тебе налили в чашку, – говорит мужик.': 'https://baneks.site'}

In [52]:
search('"бар" заходит+VERB лошадь')

{'В бар заходит лошадь.': 'https://megapanoptikum.info/tags/длинные%20анекдоты/'}

In [49]:
search('всего "два" посетителя+NOUN')

{'В баре Хогсмида всего два посетителя, один спьяну спотыкается о ногу другого.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566'}

In [50]:
search('расскажите+VERB о "героине"')

{'Теперь расскажите о героине.': 'https://baneks.site'}

In [53]:
search('NOUN VERB NOUN')

{'На здании висела вывеска: «вылечить недуг 5000 рублей, если не получится, мы вернем вам 10 000».': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Подходя к дому, видит: на крыльце сидит кот, его ждет и радостно мурлычет.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Через два часа звонит жене:\n— Кот дома?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Маугли посидел ножками поболтал и спрашивает:\n— Каа, а сильный Балу до того банана достанет?': 'https://www.anecdote.tv/no-category/6880-dlinnye-smeshnye-anekdoty-122-shtuki.html?ysclid=lnkqe5fh33111440614',
 'Мужчина приходит домой и с порога зовёт тёщу:\n– Налейте мне, мама, водочки 100 грамм.': 'https://www.anecdote.tv/no-category/6880-dlinnye-smeshnye-anekdoty-122-shtuki.html?ysclid=lnkqe5fh33111440614',
 '– Вот ещё тысяча…\nТёща взяла деньги и налила водки.': 'https://www.anecdote.tv/no

In [72]:
search('NOUN ADVB VERB')

{'Через минуту опять подходит Чебурашка и спрашивает:\n— Гена, а у тебя апельсины есть?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Через минуту опять подходит Чебурашка и спрашивает:\n— Гена, а у тебя есть молоток?': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Доктор ужасно разозлился, но оплатил счет, однако через несколько дней вернулся в клинику:\n– Здравствуйте, я потерял память, помогите мне…\n– Сестра, принесите, пожалуйста, упаковку 22 с сиропом и дайте его пациенту.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Еще более расстроенный врач заплатил, но через пару дней снова решил отыграться:\n– Здравствуйте, я потерял зрение, пожалуйста, помогите мне…\n– Мне очень жаль, но тут мы бессильны.': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566',
 'Через месяц опять приходит к врачу и говорит

In [83]:
search('врач+NOUN "опять" раскрывает+VERB')

{'Врач опять раскрывает огромный запыленный талмуд и читает:\n— Древняя китайская мудрость гласит: «Если вам отрезали оба уха, и это не помогло — значит уши здесь были ни при чём»…': 'https://anekdot-book.com/podborka/dlinnye-i-smeshnye-anekdoty?ysclid=lnm4aafl7d270279566'}